In [1]:
import pandas as pd
import urllib.request
import zipfile

In [8]:
# Data Sources
urls = ['https://www.bls.gov/oes/special.requests/oesm17ma.zip', 
'https://www.bls.gov/oes/special.requests/oesm16ma.zip',
'https://www.bls.gov/oes/special.requests/oesm15ma.zip',
'https://www.bls.gov/oes/special.requests/oesm14ma.zip',
'https://www.bls.gov/oes/special.requests/oesm13ma.zip',
'https://www.bls.gov/oes/special.requests/oesm12ma.zip',
'https://www.bls.gov/oes/special.requests/oesm11ma.zip',
'https://www.bls.gov/oes/special.requests/oesm10ma.zip',
'https://www.bls.gov/oes/special.requests/oesm09ma.zip',
'https://www.bls.gov/oes/special.requests/oesm08ma.zip']

files = ['data/raw/oesm17ma.zip', 
'data/raw/oesm16ma.zip',
'data/raw/oesm15ma.zip',
'data/raw/oesm14ma.zip',
'data/raw/oesm13ma.zip',
'data/raw/oesm12ma.zip',
'data/raw/oesm11ma.zip',
'data/raw/oesm10ma.zip',
'data/raw/oesm09ma.zip',
'data/raw/oesm08ma.zip']

In [9]:
def getData(urls, files):
    for i, j in zip(urls, files):
        urllib.request.urlretrieve(i, j)
        with zipfile.ZipFile(j) as oeszip:
            oeszip.extractall(path = 'data/raw/')            
getData(urls, files)  

In [12]:
def prepData():
    def prepData2008():
        # Obtain Data
        file200801 = "./Data/raw/MSA__M2008_dl_1.xls"
        file200802 = "./Data/raw/MSA_M2008_dl_2.xls"
        file200803 = "./Data/raw/MSA_M2008_dl_3.xls"

        x200801 = pd.ExcelFile(file200801)
        x200802 = pd.ExcelFile(file200802)
        x200803 = pd.ExcelFile(file200803)

        df200801 = x200801.parse('MSA_dl_1')
        df200802 = x200802.parse('MSA_dl_2')
        df200803 = x200803.parse('MSA_dl_3')

        df2008 = [df200801, df200802, df200803]
        df2008 = pd.concat(df2008)
        
        allOccupations = df2008[(df2008.OCC_CODE == '00-0000')]
        return allOccupations, df2008
    allOccupations, df2008 = prepData2008()
    return allOccupations, df2008
allOccupations, df2008 = prepData()
    

In [83]:
def prepDataMSA(oes):
    cols = ['PRIM_STATE', 'AREA', 'AREA_NAME', 'OCC_CODE', 'OCC_TITLE',
            'TOT_EMP', 'EMP_PRSE', 'JOBS_1000']
    oes = (oes[(oes.OCC_CODE == '15-0000')]
           .drop(oes.columns.difference(cols), 1, inplace=False)
           .replace("**", pd.np.nan).dropna(axis=0, how = "any", subset = cols)
           .apply(pd.to_numeric, errors = "ignore")
           .assign(SE = lambda x: x.EMP_PRSE * x.JOBS_1000 / 100)
           .assign(LOWER_CI = lambda x: x.JOBS_1000 - 1.96 * x.SE,
                   UPPER_CI = lambda x: x.JOBS_1000 + 1.96 * x.SE)
           .nlargest(10, 'JOBS_1000')
           .rename(columns = {'AREA_NAME':'Area',
                              'PRIM_STATE':'State',
                              'AREA':'Area Code',
                              'OCC_CODE':'OCC',
                              'OCC_TITLE':'Occupation',
                              'JOBS_1000':'C&M Jobs per 1000'})
           .set_index(['Area']))
    return oes
oes = prepData(oesData)
oes

,State,Area Code,OCC,Occupation,TOT_EMP,EMP_PRSE,C&M Jobs per 1000,SE,LOWER_CI,UPPER_CI
Area,,,,,,,,,,
"San Jose-Sunnyvale-Santa Clara, CA",CA,41940,15-0000,Computer and Mathematical Occupations,132140.0,3.1,121.332,3.761292,113.959868,128.704132
"California-Lexington Park, MD",MD,15680,15-0000,Computer and Mathematical Occupations,5510.0,22.2,121.168,26.899296,68.445380,173.890620
"San Francisco-Redwood City-South San Francisco, CA Metropolitan Division",CA,41884,15-0000,Computer and Mathematical Occupations,93670.0,4.0,83.902,3.356080,77.324083,90.479917
"Seattle-Bellevue-Everett, WA Metropolitan Division",WA,42644,15-0000,Computer and Mathematical Occupations,128040.0,6.2,77.725,4.818950,68.279858,87.170142
"Framingham, MA NECTA Division",MA,73104,15-0000,Computer and Mathematical Occupations,13320.0,4.7,77.019,3.619893,69.924010,84.113990
"Boulder, CO",CO,14500,15-0000,Computer and Mathematical Occupations,13740.0,4.3,76.986,3.310398,70.497620,83.474380
"Washington-Arlington-Alexandria, DC-VA-MD-WV Metropolitan Division",DC,47894,15-0000,Computer and Mathematical Occupations,193830.0,1.6,76.940,1.231040,74.527162,79.352838
"Huntsville, AL",AL,26620,15-0000,Computer and Mathematical Occupations,14290.0,4.4,64.339,2.830916,58.790405,69.887595
"Lowell-Billerica-Chelmsford, MA-NH NECTA Division",MA,74804,15-0000,Computer and Mathematical Occupations,9800.0,13.2,62.973,8.312436,46.680625,79.265375
